In [ ]:
!pip install -q timm
import timm
import cv2
import glob
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import albumentations as A
from albumentations.pytorch import ToTensorV2
import json
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
import time
import datetime

In [ ]:
LABEL_SMOOTHING = True
LR_SCHEDULING = True
TTA = True
MODEL_ARCHITECTURE = 'vit_base_patch16_384'

## 1. Tìm hiểu dữ liệu (Exploratory data analysis)

Đọc các file dữ liệu cho bài toán:
* `label_num_to_disease_map.json`: dictionary lưu id bệnh (nhãn) - tên bệnh.
* `train.csv`: file csv chứa thông tin tên file ảnh và nhãn (id bệnh) tương ứng.
* `train_images/*`: directory chứa các file ảnh.

In [ ]:
with open('../input/cassava-leaf-disease-classification/label_num_to_disease_map.json', 'r') as f:
    label_names = json.load(f)
annotations = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
train_img_dir = '../input/cassava-leaf-disease-classification/train_images'

Có 4 loại bệnh khác nhau, label từ 0 -> 3, label 4 là để chỉ lá khỏe mạnh bình thường

In [ ]:
label_names

In [ ]:
print(f'Tập train có {len(annotations)} ảnh.')
annotations.head()

Độ phân giải của ảnh: tất cả các ảnh trong tập train đều có độ phân giải là $600 \times 800 \times 3$ ($h \times w \times c$)

In [ ]:
def check_resolution(img_dir):
    """
    Kiểm tra tất cả ảnh trong directory có những độ phân giải nào
    Args:
        img_dir: string - đường dẫn directory chứa file ảnh
    Returns:
        set of tuple - tập tất cả các độ phân giải khác nhau, 
            mỗi độ phân giải là 1 tuple (height, width, channels)
    """
    resolutions = set()
    img_paths = glob.glob(os.path.join(img_dir, '*.jpg'))
    for img_path in tqdm(img_paths):
        resolutions.add(cv2.imread(img_path).shape)
    return resolutions

In [ ]:
# resolutions = check_resolution(train_img_dir)
# print(resolutions)

Kiểm tra phân phối các nhãn trong tập train

In [ ]:
label_counts = annotations['label'].astype(str).map(label_names).value_counts()
plt.figure(figsize=(8, 8))
plt.pie(label_counts, labels=label_counts.index, autopct='%1.1f%%')
plt.title('Phân phối nhãn trong tập train')
plt.axis('equal')
plt.show()

$\rightarrow$ Có thể thấy phân phối các nhãn không cân bằng. Cassava Mosaic Disease chiếm hơn một nửa trong bộ dữ liệu

Visualize một vài ảnh ví dụ trong tập train cùng với nhãn tương ứng

In [ ]:
def visualize(label, img_dir, annotations, label_names):
    """
    Visualize một ảnh trong tập train cùng với label tương ứng của nó
    """
    img_id = annotations[annotations['label']==label].iloc[0]['image_id']
    img_path = os.path.join(img_dir, img_id)
    img = cv2.imread(img_path) # BGR format
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # BGR -> RGB
    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.title(label_names[str(label)])
    plt.show()

In [ ]:
for label in range(5):
    visualize(label, train_img_dir, annotations, label_names)

Có thể thấy dữ liệu có nhiễu (đánh nhãn sai). Ảnh cuối ở trên không thể nào thuộc nhãn `Healthy` được

## 2. Tiền xử lý dữ liệu (Data preprocess)

## a. Load dataset
https://pytorch.org/tutorials/recipes/recipes/custom_dataset_transforms_loader.html

Tiền xử lý dữ liệu ảnh đầu vào:
1. Đọc dữ liệu ảnh và lưu dưới dạng numpy array sử dụng opencv
2. Resize về kích thước cố định (chẳng hạn 512x512)
3. Rescale pixel values từ [0, 255] về [0, 1] rồi normalize với rgb mean và std của dataset này (hoặc của ImageNet dataset).
4. Convert numpy array thành torch tensor làm input cho cnn model

Đối với dữ liệu train có thể sử dụng data augmentation (làm giàu dữ liệu) để cải thiện hiệu quả:
* Basic augmentation: random resize + crop, horizontal/vertical flip, shift + scale + rotate, adjust HSV/RGB values, adjust brightness/contrast, v.v.
* Advanced augmentation: Cutout, Mixup, Cutmix, v.v. 

Test time augmentation: thực hiện nhiều augmentation lúc inference và lấy kết quả trung bình

In [ ]:
class CassavaDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, img_size, rgb_mean, rgb_std, 
                 augment=False, tta_idx=None, annotations=None):
        """
        Args:
            img_dir: string - đường dẫn directory chứa ảnh
            img_size: int - kích thước ảnh muốn resize
            rgb_mean: tuple (r, g, b) - kì vọng của từng channel (dùng để chuẩn hóa)
            rgb_std: tuple (r, g, b) - độ lệch chuẩn của từng channel (dùng để chuẩn hóa)
            augment: boolean - có làm giàu dữ liệu hay không
            tta_idx: int 0->4 hoặc None - test time augmentation, thực hiện phép biến đổi nào,
                                          =None thì biến đổi random (train)
                                          chỉ có tác dụng khi augment=True
            annotations: pd.DataFrame object - dataframe chứa thông tin label của ảnh
                                               dùng cho tập train/validate (= None khi inference)
        """
        self.img_dir = img_dir
        self.annotations = annotations
        self.img_ids = None
        if self.annotations is None:
            self.img_ids = os.listdir(self.img_dir)
            
        # 5 phép biến đổi tta:
        # 0. Resize về img_size x img_size (dùng lúc validate)
        # 1. Crop img_size x img_size ở chính giữa ảnh
        # 2. Crop rồi lật ngang
        # 3. Crop rồi lật dọc
        # 4. Crop rồi chuyển vị
        tta_transform_lists = [
            [A.Resize(img_size, img_size)],
            [A.CenterCrop(img_size, img_size, p=1.0)],
            [
                A.CenterCrop(img_size, img_size, p=1.0),
                A.HorizontalFlip(p=1.0)
            ],
            [
                A.CenterCrop(img_size, img_size, p=1.0),
                A.VerticalFlip(p=1.0)
            ],
            [
                A.CenterCrop(img_size, img_size, p=1.0),
                A.Transpose(p=1.0)
            ]
        ]
            
        if augment:
            # Test time augmentation cho lúc test
            if tta_idx is not None:
                transform_list = tta_transform_lists[tta_idx] + [
                    A.Normalize(mean=rgb_mean, std=rgb_std, 
                                max_pixel_value=255.0, p=1.0),
                    ToTensorV2(p=1.0) # (h, w, c) -> (c, h, w)
                ]
            # Random augmentation cho lúc train
            else:
                transform_list = [
                    A.RandomResizedCrop(img_size, img_size,
                                        ratio=(0.5, 2.0)),
                    A.Transpose(p=0.5),
                    A.HorizontalFlip(p=0.5),
                    A.VerticalFlip(p=0.5),
                    A.ShiftScaleRotate(p=0.5),
                    A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, 
                                         val_shift_limit=0.2, p=0.5),
                    A.RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), 
                                               contrast_limit=(-0.1, 0.1), p=0.5),
                    A.Normalize(mean=rgb_mean, std=rgb_std, 
                                max_pixel_value=255.0, p=1.0),
                    A.CoarseDropout(p=0.5),
                    A.Cutout(p=0.5),
                    ToTensorV2(p=1.0) # (h, w, c) -> (c, h, w)
                ]
        else:
            transform_list = tta_transform_lists[0] + [
                A.Normalize(mean=rgb_mean, std=rgb_std, 
                            max_pixel_value=255.0, p=1.0),
                ToTensorV2(p=1.0) # (h, w, c) -> (c, h, w)
            ]
        self.transform = A.Compose(transform_list)
    
    def __len__(self):
        if self.annotations is not None:
            return len(self.annotations)
        else:
            self.img_ids = os.listdir(self.img_dir)
            return len(self.img_ids)
        
    def load_image(self, img_id):
        img = cv2.imread(os.path.join(self.img_dir, img_id)) # (h, w, c) in BGR format
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # BGR -> RGB
        img = self.transform(image=img)['image'] # augment (optional), 
                                                 # normalized and rescaled to [0, 1], 
                                                 # shape (3, img_size, img_size)
        return img
    
    def __getitem__(self, idx):
        if self.annotations is not None:
            row = self.annotations.iloc[idx]
            img_id, label = row['image_id'], row['label']
            img = self.load_image(img_id)
            return img, label
        else:
            img_id = self.img_ids[idx]
            img = self.load_image(img_id)
            return img, img_id

## b. Stratified K-fold split

* Chia tập dữ liệu train thành 5 fold
* Số lượng dữ liệu của các class không cân bằng $\rightarrow$ sử dụng stratified sampling
* Stratified sampling: phương pháp chia đảm bảo tỉ lệ các class trong mỗi tập con vẫn giữ nguyên/gần giống so với tập gốc. https://en.wikipedia.org/wiki/Stratified_sampling

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds = skf.split(annotations['image_id'].values, annotations['label'].values)
for i, (train_indices, val_indices) in enumerate(folds):
    annotations.loc[val_indices, 'fold'] = i
annotations['fold'] = annotations['fold'].astype(int)

dataframe bây giờ có thêm cột `fold` để chỉ ra mỗi sample thuộc fold nào

In [ ]:
annotations.head()

In [ ]:
# kích thước ảnh resize để cho vào mạng CNN
img_size = 384 # 512

# RGB mean and std of this dataset
# rgb_mean = (0.43032043, 0.49672632, 0.31342008)
# rgb_std = (0.21909042, 0.22394303, 0.20059191)

# RGB mean and std of Imagenet dataset
rgb_mean = (0.485, 0.456, 0.406)
rgb_std = (0.229, 0.224, 0.225)

# chọn fold để làm tập validation, các fold còn lại là tập train
fold = 1

# chia dataframe thành train-validation
train_annotations = annotations[annotations['fold']!=fold].reset_index(drop=True)
val_annotations = annotations[annotations['fold']==fold].reset_index(drop=True)

# Đối với tập train thì thực hiện làm giàu dữ liệu, còn validate thì không cần
train_dataset = CassavaDataset(img_dir=train_img_dir, 
                               img_size=img_size,
                               rgb_mean=rgb_mean, 
                               rgb_std=rgb_std,
                               augment=True,
                               tta_idx=None,
                               annotations=train_annotations)

val_dataset = CassavaDataset(img_dir=train_img_dir,
                             img_size=img_size,
                             rgb_mean=rgb_mean,
                             rgb_std=rgb_std,
                             augment=False,
                             tta_idx=None,
                             annotations=val_annotations)

## c. DataLoader
Thực hiện shuffle dữ liệu, gộp dữ liệu thành các batch, v.v.

In [ ]:
batch_size = 16 # số sample trong một batch

# Tập train thì cần xáo (shuffle) dữ liệu để đảm bảo tính ngẫu nhiên, còn tập validate thì không cần
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                           shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, 
                                         shuffle=False, num_workers=4)

## 3. Model
Transfer learning: sử dụng mô hình resnet18 pretrained với Imagenet dataset để finetune cho bài toán phân loại bệnh lá sắn
https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html#finetuning-the-convnet

Note:
* Để giữ output shape không thay đổi so với input shape khi đi qua convolution filter ("same padding") thì $$2 \cdot padding = kernel\_size - 1$$ (với điều kiện dilation và stride để mặc định bằng 1)
* Hàm `torch.nn.CrossEntropyLoss()` không tính cross entropy giữa output và target ngay mà trước hết tính softmax của output để convert thành dạng probabilities, rồi sau đó mới tính cross entropy giữa softmax(output) và target. Do đó nếu dùng `torch.nn.CrossEntropyLoss()` thì lúc tạo CNN không cần phải dùng `torch.nn.Softmax()` ở layer cuối cùng. (Output raw của neural network chưa đi qua softmax còn hay được gọi là ["logits"](https://developers.google.com/machine-learning/glossary/#logits)). Tham khảo: https://stackoverflow.com/a/49839941

In [ ]:
n_classes = len(label_names) # số class = 5 (4 loại bệnh + 1 bình thường)

# Load mô hình với weights đã được pretrain trên ImageNet dataset
model = timm.create_model(MODEL_ARCHITECTURE, 
                          pretrained=True, 
                          num_classes=n_classes,
                          checkpoint_path='')

## 4. Optimizer, Loss function, Utilities

* Như đã đề cập ở trên, mô hình output ra logits (chưa đi qua lớp softmax) $\rightarrow$ dùng loss function `nn.CrossEntropyLoss()`
* Thuật toán tối ưu sử dụng là Stochastic Gradient Descent (hoặc Adam), các biến cần tối ưu là các tham số của mô hình
* Learning rate scheduler `StepLR`: cứ mỗi `step_size` step thì nhân learning rate với `gamma`
* Label smoothing: nhãn one-hot (1, 0, 0, 0, 0) $\rightarrow$ (0.8, 0.05, 0.05, 0.05, 0.05), tránh overfit, hạn chế ảnh hưởng dữ liệu nhiễu

In [ ]:
def smooth_one_hot(true_labels: torch.Tensor, classes: int, smoothing=0.0):
    """
    if smoothing == 0, it's one-hot method
    if 0 < smoothing < 1, it's smooth method

    """
    assert 0 <= smoothing < 1
    confidence = 1.0 - smoothing
    label_shape = torch.Size((true_labels.size(0), classes))
    with torch.no_grad():
        true_dist = torch.empty(size=label_shape, device=true_labels.device)
        true_dist.fill_(smoothing / (classes - 1))
        true_dist.scatter_(1, true_labels.data.unsqueeze(1), confidence)
    return true_dist


class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        true_dist = smooth_one_hot(target, self.cls, self.smoothing)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

if LABEL_SMOOTHING:
    # label smoothing với hệ số smoothing 0.2
    loss_func = LabelSmoothingLoss(classes=n_classes, smoothing=0.2)
else:
    loss_func = nn.CrossEntropyLoss()
    
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
scheduler = None
if LR_SCHEDULING:
    # giảm lr 5 lần mỗi 5 epoch
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.2)

Một số class và hàm dùng để track running loss/accuracy trong quá trình train:
[(reference)](https://github.com/rwightman/pytorch-image-models/blob/master/timm/utils/metrics.py)

In [ ]:
# https://github.com/rwightman/pytorch-image-models/blob/master/timm/utils/metrics.py
class AverageMeter:
    """
    Computes and stores the average and current value
    Use this for losses with reduction='mean'
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
def accuracy(output, target, topk=(1,)):
    """
    Computes the accuracy over the k top predictions for the specified values of k
    """
    maxk = max(topk)
    batch_size = target.size(0)
    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.reshape(1, -1).expand_as(pred))
    return [correct[:k].reshape(-1).float().sum(0) * 100. / batch_size for k in topk]

## 5. Training loop

Note
* Một số loại layer như **dropout** hay **batchnorm** sẽ làm cho behavior của model khi train và khi test khác nhau. Sử dụng `model.train()` và `model.eval()` để thay đổi mode của model theo ý muốn.
https://stackoverflow.com/a/51433411
* Các bước để train một batch:
 1. Cho batch ảnh đi qua model $\rightarrow$ output là logits
 2. Tính cross entropy loss của `softmax(logits)` và batch nhãn
 3. Lan truyền ngược để tính gradient loss theo tất cả các tham số mô hình
 4. Cập nhật tất cả tham số dựa trên gradient tính ở bước trên
* Mặc định mỗi lần gọi `.backward()` để tính gradient thì pytorch sẽ **tích lũy** gradient ở các node lá. Do đó trước đấy ta cần phải gọi `zero_grad()` để reset tất cả gradient về 0. https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch

In [ ]:
def train_epoch(model, loader, device, loss_func, optimizer, verbose):
    """
    Train mô hình một epoch
    Args:
        model: model cần train
        loader: torch dataloader để load dữ liệu train
        device: torch device (gpu hoặc cpu)
        loss_func: hàm tổn thất
        optimizer: thuật toán tối ưu sử dụng
        verbose: boolean - có in đầy đủ thông tin trong quá trình train hay không
    Returns:
        summary_loss, summary_acc
    """
    model.train() # chuyển mode sang 'train' (dùng cho dropout, batchnorm, v.v.)
    summary_loss = AverageMeter() # track running loss trong quá trình train
    summary_acc = AverageMeter() # track running accuracy trong quá trình train
    start = time.time() # track thời gian train
    
    n = len(loader) # số batch một epoch
    # mỗi batch của loader là một tuple (img_batch, label_batch)
    # img_batch: torch tensor, shape (batch_size, 3, img_size, img_size)
    # label_batch: torch tensor, shape (batch_size,)
    for step, (img_batch, label_batch) in enumerate(loader):
        batch_size = img_batch.size(0)
        
        # Chuyển dữ liệu đến gpu (nếu có)
        img_batch = img_batch.to(device)
        label_batch = label_batch.to(device)
        
        # output của model là logits
        # shape là (batch_size, n_classes)
        out_batch = model(img_batch)
        # Tính loss của batch này
        loss = loss_func(out_batch, label_batch)
        
        # Thực hiện lan truyền ngược (backpropagation)
        # 1. Reset gradient của tất cả tham số mô hình về 0
        optimizer.zero_grad()
        # 2. Tính gradient loss đối với tất cả tham số mô hình
        loss.backward()
        # 3. Cập nhật tất cả các tham số mô hình dựa vào các gradient tính ở trên
        optimizer.step()
        
        # Tính accuracy cho batch này (nếu ko cần track gradient thì gọi `torch.no_grad()`)
        with torch.no_grad():
            acc = accuracy(out_batch, label_batch)[0]
        
        # Update running loss và running accuracy
        summary_loss.update(loss.item(), batch_size)
        summary_acc.update(acc.item(), batch_size)
        if verbose:
            print('Train step {}/{}, loss: {:.5f}'.format(step + 1, n, summary_loss.avg), end='\r')
    train_time = str(datetime.timedelta(seconds=time.time() - start))
    print('Train loss: {:.5f} - Train acc: {:.2f}% - time: {}'.format(summary_loss.avg, 
                                                                      summary_acc.avg,
                                                                      train_time))
    return summary_loss, summary_acc
        
def evaluate_epoch(model, loader, device, loss_func, verbose):
    """
    Evaluate mô hình
    Args:
        model: model cần evaluate
        loader: torch dataloader để load dữ liệu validation
        device: torch device (gpu hoặc cpu)
        loss_func: hàm tổn thất
        verbose: boolean - có in đầy đủ thông tin trong quá trình evaluate hay không
    Returns:
        summary_loss, summary_acc
    """
    model.eval() # chuyển mode sang 'evaluate' (dùng cho dropout, batchnorm, v.v.)
    summary_loss = AverageMeter() # track running loss trong quá trình evaluate
    summary_acc = AverageMeter() # track running accuracy trong quá trình evaluate
    start = time.time() # track thời gian evaluate
    
    n = len(loader) # số batch một epoch
    # mỗi batch của loader là một tuple (img_batch, label_batch)
    # img_batch: torch tensor, shape (batch_size, 3, img_size, img_size)
    # label_batch: torch tensor, shape (batch_size,)
    for step, (img_batch, label_batch) in enumerate(loader):
        with torch.no_grad():
            batch_size = img_batch.size(0)
            
            # Chuyển dữ liệu đến gpu (nếu có)
            img_batch = img_batch.to(device)
            label_batch = label_batch.to(device)

            # output của model là logits
            # shape là (batch_size, n_classes)
            out_batch = model(img_batch)
            # Tính loss của batch này
            loss = loss_func(out_batch, label_batch)
            # Tính accuracy cho batch này
            acc = accuracy(out_batch, label_batch)[0]
            
            # Update running loss và running accuracy
            summary_loss.update(loss.detach().item(), batch_size)
            summary_acc.update(acc.detach().item(), batch_size)
            if verbose:
                print('Val step {}/{}'.format(step + 1, n), end='\r')
    eval_time = str(datetime.timedelta(seconds=time.time() - start))
    print('Val loss: {:.5f} - Val acc: {:.2f}% - time: {}'.format(summary_loss.avg,
                                                                  summary_acc.avg,
                                                                  eval_time))
    return summary_loss, summary_acc

Mỗi vòng lặp train thực hiện các công việc:
1. Train 1 epoch trên training set
2. Evaluate trên validation set $\rightarrow$ tính loss, metrics trên tập validation
3. Nếu performance trên tập validation (loss hoặc accuracy ...) đạt kỷ lục mới thì save model

In [ ]:
# mặc định sử dụng GPU, nếu không thì sử dụng CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# một số biến để track kỉ lục hiện tại
best_acc = -np.inf
best_epoch = 0

verbose = False # có in đầy đủ thông tin không
epochs = 15 # số epoch

# lưu mô hình
save_path = '{}_{}'.format(MODEL_ARCHITECTURE, img_size)
if LABEL_SMOOTHING:
    save_path += '_ls'
if LR_SCHEDULING:
    save_path += '_lrs'
save_path += '_fold{}.pth'.format(fold)

for epoch in range(epochs):
    print('Epoch {}/{}:'.format(epoch + 1, epochs))
    print('-' * 10)
    train_loss, train_acc = train_epoch(model, train_loader, device, loss_func, optimizer, verbose)
    val_loss, val_acc = evaluate_epoch(model, val_loader, device, loss_func, verbose)
    
    # điều chỉnh learning rate
    if LR_SCHEDULING:
        scheduler.step()
        
    # nếu validation accuracy đạt kỉ lục mới thì save model
    if val_acc.avg > best_acc:
        best_acc = val_acc.avg
        best_epoch = epoch
        print('Saving model...')
        torch.save(model.state_dict(),
                   save_path)
print('Finished. Best val_acc achieved is {:.2f}% at epoch {}'.format(best_acc, best_epoch))

## 6. Inference

Kết quả inference cho mỗi ảnh: 
* `probabilities` là xác suất dự đoán từng nhãn
* `label` là nhãn dự đoán (nhãn có xác suất cao nhất)

In [ ]:
def inference(model, loader, device):
    """
    Inference với mô hình đã train
    Args:
        model: model đã được train
        loader: torch dataloader để load dữ liệu test
        device: torch device (gpu hoặc cpu)
    Returns:
        dataframe chứa kết quả inference
    """
    model.eval() # chuyển mode sang 'evaluate' (dùng cho dropout, batchnorm, v.v.)
    preds = [] # lưu kết quả predict
    img_ids = [] # lưu các image id
    n = len(loader)
    
    # mỗi batch của loader là một tuple (img_batch, img_id_batch)
    # img_batch: torch tensor, shape (batch_size, 3, img_size, img_size)
    # img_id_batch: một list các string dưới dạng torch tensor (batch_size,)
    for step, (img_batch, img_id_batch) in enumerate(loader):
        # khi test không cần tính gradient
        with torch.no_grad():
            # Chuyển dữ liệu đến gpu (nếu có)
            img_batch = img_batch.to(device)
            
            # output của model là logits
            # sau khi cho đi qua lớp softmax sẽ thành probabilities
            # shape là (batch_size, n_classes)
            out_batch = nn.functional.softmax(model(img_batch), dim=1)
            
            # thêm các kết quả trong batch vào list tổng
            preds.append(out_batch.detach().cpu().numpy())
            img_ids += list(img_id_batch)
            
            print('Inference step {}/{}'.format(step + 1, n), end='\r')
    print('\n')
            
    preds = np.concatenate(preds, axis=0) # convert list sang np.array, shape (n_samples, n_classes)
    # lưu kết quả vào một dataframe
    result = pd.DataFrame({'image_id': img_ids, 
                           'probabilities': list(preds)})
    return result

## a. Evaluate và visualize trên tập validation

In [ ]:
# load best model weights
model.load_state_dict(torch.load(save_path))

eval_loaders = []

if TTA:
    for tta_idx in range(5):
        eval_dataset = CassavaDataset(img_dir=train_img_dir,
                                      img_size=img_size,
                                      rgb_mean=rgb_mean,
                                      rgb_std=rgb_std,
                                      augment=True,
                                      tta_idx=tta_idx,
                                      annotations=val_annotations)
        eval_loaders.append(torch.utils.data.DataLoader(eval_dataset, batch_size=batch_size, 
                                                        shuffle=False, num_workers=4))
else:
    eval_dataset = CassavaDataset(img_dir=train_img_dir,
                                  img_size=img_size,
                                  rgb_mean=rgb_mean,
                                  rgb_std=rgb_std,
                                  augment=False,
                                  tta_idx=None,
                                  annotations=val_annotations)
    eval_loaders.append(torch.utils.data.DataLoader(eval_dataset, batch_size=batch_size, 
                                                    shuffle=False, num_workers=4))

Visualize một số ảnh và dự đoán trên tập validation

In [ ]:
eval_results = [inference(model, eval_loader, device)[['probabilities']]
               for eval_loader in eval_loaders]
eval_results = [eval_result.rename(columns={'probabilities': f'probabilities_{i}'})
               for i, eval_result in enumerate(eval_results)] 

In [ ]:
eval_result = pd.concat(eval_results, axis=1)
eval_result['probabilities'] = eval_result.sum(axis=1) / len(eval_result.columns)
eval_result['pred'] = eval_result['probabilities'].map(lambda x: np.argmax(x)) # probability to class
eval_result = eval_result[['probabilities', 'pred']]
eval_result['image_id'] = val_annotations['image_id']

In [ ]:
eval_result = pd.merge(eval_result, val_annotations[['image_id', 'label']], on='image_id')
eval_acc = (eval_result['pred'] == eval_result['label']).sum() / len(eval_result)
print('Validation accuracy: {:.2f}%'.format(100*eval_acc))

In [ ]:
def visualize_prediction(img_dir, eval_result, label_names):
    """
    Visualize một ảnh ngẫu nhiên trong tập validation và dự đoán tương ứng
    """
    random_row = eval_result.sample()
    img_id = random_row['image_id'].item() 
    label = random_row['label'].item() 
    pred = random_row['pred'].item()
    probabilities = random_row['probabilities'].item()
    
    img_path = os.path.join(img_dir, img_id)
    img = cv2.imread(img_path) # BGR format
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # BGR -> RGB
    
    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    plt.axis('off')
    title = 'Nhãn: {}\nDự đoán: {}, xác suất: {:.2f}%'.format(label_names[str(label)],
                                                              label_names[str(pred)],
                                                              100*probabilities[pred])
    plt.title(title)

In [ ]:
for _ in range(5):
    visualize_prediction(train_img_dir, eval_result, label_names)

## b. Inference trên tập test

Đối với tập test:
* `augment=False`: không làm giàu dữ liệu
* `annotations=None`: không có nhãn trong tập test, mỗi sample trong tập test thay vì trả về `(ảnh, label)` thì sẽ trả về `(ảnh, id ảnh)`
* dataloader `shuffle=False`: không cần phải xáo dữ liệu

In [ ]:
test_img_dir = '../input/cassava-leaf-disease-classification/test_images'

test_loaders = []

if TTA:
    for tta_idx in range(5):
        test_dataset = CassavaDataset(img_dir=test_img_dir,
                                      img_size=img_size,
                                      rgb_mean=rgb_mean,
                                      rgb_std=rgb_std,
                                      augment=True,
                                      tta_idx=tta_idx,
                                      annotations=None)
        test_loaders.append(torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, 
                                                        shuffle=False, num_workers=4))
else:
    test_dataset = CassavaDataset(img_dir=test_img_dir,
                                  img_size=img_size,
                                  rgb_mean=rgb_mean,
                                  rgb_std=rgb_std,
                                  augment=False,
                                  tta_idx=None,
                                  annotations=None)
    test_loaders.append(torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, 
                                                    shuffle=False, num_workers=4))

In [ ]:
test_results = [inference(model, test_loader, device).set_index('image_id')
               for test_loader in test_loaders]
test_results = [test_result.rename(columns={'probabilities': f'probabilities_{i}'})
               for i, test_result in enumerate(test_results)] 

test_result = pd.concat(test_results, axis=1)
test_result['probabilities'] = test_result.sum(axis=1) / len(test_result.columns)
test_result['label'] = test_result['probabilities'].map(lambda x: np.argmax(x)) # probability to class
test_result

## c. Submission

Kiểm tra lại format submission giống với file `sample_submission.csv`

In [ ]:
sample_submission = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
sample_submission

In [ ]:
test_result[['label']].reset_index()

Sau khi kiểm tra cẩn thận, lưu file submission vào file `submission.csv` (phải đặt ở directory `/kaggle/working/`)

In [ ]:
test_result[['label']].reset_index().to_csv('submission.csv', index=False)